In [1]:
from __future__ import print_function
import sys
import SimpleITK as sitk
import dicom, os, copy, glob
from dicom.tag import Tag
import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import polygon
from multiprocessing import cpu_count, Pool
from numba import jit, cuda
import copy
from copy import deepcopy
import re
import pandas as pd
import numba
import radiomics
import collections
from radiomics import featureextractor 
from radiomics import firstorder, getTestCase, glcm, glrlm, glszm, imageoperations, shape, generalinfo
from radiomics import _cmatrices as cMatrices
from radiomics import _cshape as cShape
from skimage.draw import polygon
import dicom
from dicom_contour.contour import *
from skimage.draw import polygon
from pydicom import dcmread
from __future__ import print_function
import six

C:\Application\anaconda3\envs\Radiomics\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Application\anaconda3\envs\Radiomics\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Application\anaconda3\envs\Radiomics\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Application\anaconda3\envs\Radiomics\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Application\anaconda3\envs\Radiomics\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Application\anaconda3\envs\Radiomics\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Application\anaconda3\envs\Radiomics\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1

In [2]:
def read_structure(structure):
    contours = []
    for i in range(len(structure.ROIContourSequence)):
        contour = {}
        contour['color'] = structure.ROIContourSequence[i].ROIDisplayColor
        contour['number'] = structure.ROIContourSequence[i].ReferencedROINumber
        contour['name'] = structure.StructureSetROISequence[i].ROIName
        assert contour['number'] == structure.StructureSetROISequence[i].ROINumber
        try:
            contour['contours'] = [s.ContourData for s in structure.ROIContourSequence[i].ContourSequence]
        except:
            pass
        contours.append(contour)
    return contours

def get_mask(contours, slices, image):
    z = [s.ImagePositionPatient[2] for s in slices]
    pos_r = slices[0].ImagePositionPatient[1]
    spacing_r = slices[0].PixelSpacing[1]
    pos_c = slices[0].ImagePositionPatient[0]
    spacing_c = slices[0].PixelSpacing[0]
    label = np.zeros_like(image, dtype=np.uint8)
    
    for c in contours["contours"]:
        nodes = np.array(c).reshape((-1, 3))
        assert np.amax(np.abs(np.diff(nodes[:, 2]))) == 0
        z_index = z.index(nodes[0, 2])
        r = (nodes[:, 1] - pos_r) / spacing_r
        c = (nodes[:, 0] - pos_c) / spacing_c
        rr, cc = polygon(abs(r), abs(c))
        #rr, cc = polygon(r, c)
        label[z_index, rr, cc] = 1
        #print(rr)
        
    return label

In [3]:
# Get the testCase
filepath="C:/Users/Du/Test/Data"
ptdir = [os.path.join(filepath, name)
                   for name in os.listdir(filepath)
                   if os.path.isdir(os.path.join(filepath, name))]
subdir = ptdir[0]+'/DICOM'
print ([0,subdir])

[0, 'C:/Users/Du/Test/Data\\001/DICOM']


In [4]:
# Get image info
print("Reading Dicom directory:", subdir)
series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(subdir)
series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(subdir, series_IDs[0])
series_reader = sitk.ImageSeriesReader()
series_reader.SetFileNames(series_file_names)
series_reader.LoadPrivateTagsOn()
image3D = series_reader.Execute()
dimension=image3D.GetDimension()
size = image3D.GetSize()
origin = image3D.GetOrigin()
spacing = image3D.GetSpacing()
direction = image3D.GetDirection()

print("Image dimension:", dimension)
print("Image origin:", origin)
print("Image size:", size)
print("Image spacing:", spacing)
print("Image direction:", direction)

Reading Dicom directory: C:/Users/Du/Test/Data\001/DICOM
Image dimension: 3
Image origin: (-250.0, -169.39999389648, -1119.5)
Image size: (512, 512, 228)
Image spacing: (0.9765625, 0.9765625, 2.0)
Image direction: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


In [5]:
# Get structure info
for subdir, dirs, files in os.walk(subdir):
    dcms = series_file_names
    slices = [dicom.read_file(dcm) for dcm in dcms]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))

contour_file = get_contour_file(subdir)
contour_data = dicom.read_file(subdir + '/' + contour_file)
roi_names = get_roi_names(contour_data)
print(roi_names)

Contour_Names = 'LUNG_L'

gtv_idx=roi_names.index(Contour_Names)


ds = dcmread(subdir + '/' + contour_file)
contours = read_structure(ds)
cc = contours[gtv_idx]

image = sitk.GetArrayFromImage(image3D)
mask = get_mask(cc, slices, image)
print(mask.shape)

gtv_mask = sitk.GetImageFromArray(mask.data, sitk.sitkUInt8)

last_foldername=os.path.basename(ptdir[0])
foldernames = os.path.join(filepath + "/" + last_foldername + "/DICOM/")
filenames = os.listdir(foldernames)

['Bowel_sm_CBCT', 'LUNG_L', 'LUNG_R', 'Stomach_duo_CBCT']
(228, 512, 512)


In [6]:
# Get Dose info
RTD_name=[]
for f in filenames:
    if re.match('RTD_', f):
        RTD_name=f

reader = sitk.ImageFileReader()
reader.SetFileName(os.path.join(foldernames,RTD_name))
img = reader.Execute()

ds = dicom.read_file(os.path.join(foldernames,RTD_name))

if ds[0x30040002].value=='GY':
    img_array=sitk.GetArrayFromImage(img)
    img_array=img_array*ds[0x3004000E].value*100
if ds[0x30040002].value=='cGY':
    img_array=sitk.GetArrayFromImage(img)
    img_array=img_array*ds[0x3004000E].value
    
img1=sitk.GetImageFromArray(img_array)
img1.SetSpacing(img.GetSpacing())
img1.SetOrigin(img.GetOrigin())
img1.SetDirection(img.GetDirection())
identity = sitk.Transform(3, sitk.sitkIdentity)

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(image3D)
resampler.SetInterpolator(sitk.sitkBSpline)
resampler.SetSize(image3D.GetSize())
resampler.SetOutputOrigin(image3D.GetOrigin())
resampler.SetOutputDirection(image3D.GetDirection())
resampler.SetOutputSpacing(image3D.GetSpacing())
resampler.SetTransform(identity)
dosemap = resampler.Execute(img1)

In [7]:
# First define the settings

gtv_mask=sitk.GetImageFromArray(sitk.GetArrayFromImage(gtv_mask))
gtv_mask.SetSpacing(spacing)
gtv_mask.SetOrigin(origin)
gtv_mask.SetDirection(direction)

image3D=sitk.Cast(image3D, sitk.sitkInt16)
gtv_mask=sitk.Cast(gtv_mask, sitk.sitkUInt8)
patient_name=os.path.basename(ptdir[0])

settings = {}
settings['binWidth'] = 25
settings['sigma'] = [1.0, 3.0, 5.0]
settings['preCrop'] = True

# Resampling:
# If slices are very thin (~1mm), such as in 3D scanned (isotropic) volumes, resampledPixelSpacing may be reduced to
# (1, 1, 1). Furthermore, in case of isotropic volumes, consider disabling resampling.
# On a side note: increasing the resampled spacing forces PyRadiomics to look at more coarse textures, which may or
# may not increase accuracy and stability of your extracted features.

settings['interpolator'] = 'sitkBSpline'
settings['resampledPixelSpacing'] = [1, 1, 1]
settings['padDistance'] = 10 # Set pad distance when cropping images

In [8]:
# Instantiate the extractor

extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
print('Extraction parameters:\n\t', extractor.settings)

extractor.disableAllImageTypes()
extractor.enableImageTypeByName('Original')
extractor.enableImageTypeByName('Wavelet')
extractor.enableImageTypeByName('LoG')

extractor.enableAllFeatures()

print('Calculating features')

outputFilepath = filepath # Set you own output path to save radiomics/dosiomics features.

featureVector = extractor.execute(image3D, gtv_mask)
result = pd.DataFrame(pd.Series(featureVector)).T
result.to_csv(outputFilepath + patient_name + '_radiomics.csv', index=False, na_rep='NaN')

RTstructure='rectal_gtv'
##image, mask = imageoperations.resampleImage(sitk_imgs, sitk_dmask, settings['resampledPixelSpacing'], settings['interpolator'], label=1, padDistance=10)
bb, correctedMask = imageoperations.checkMask(image3D, gtv_mask)
if correctedMask is not None:
    gtv_mask = correctedMask
image, mask = imageoperations.cropToTumorMask(image3D, gtv_mask, bb, padDistance=10)


outputFilepath_D = filepath # Set you own output path to save radiomics/dosiomics features.

featureVector = extractor.execute(dosemap, gtv_mask)
result = pd.DataFrame(pd.Series(featureVector)).T
result.to_csv(outputFilepath_D + patient_name + '_dosiomics.csv', index=False, na_rep='NaN')


Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [1, 1, 1], 'interpolator': 'sitkBSpline', 'preCrop': True, 'padDistance': 10, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'sigma': [1.0, 3.0, 5.0]}
Calculating features


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calc

In [9]:
for key, value in six.iteritems(result):
    print('\t', key, ':', value)

	 diagnostics_Versions_PyRadiomics : 0    v3.0.1
Name: diagnostics_Versions_PyRadiomics, dtype: object
	 diagnostics_Versions_Numpy : 0    1.22.4
Name: diagnostics_Versions_Numpy, dtype: object
	 diagnostics_Versions_SimpleITK : 0    2.2.0
Name: diagnostics_Versions_SimpleITK, dtype: object
	 diagnostics_Versions_PyWavelet : 0    1.1.1
Name: diagnostics_Versions_PyWavelet, dtype: object
	 diagnostics_Versions_Python : 0    3.8.5
Name: diagnostics_Versions_Python, dtype: object
	 diagnostics_Configuration_Settings : 0    {'minimumROIDimensions': 2, 'minimumROISize': ...
Name: diagnostics_Configuration_Settings, dtype: object
	 diagnostics_Configuration_EnabledImageTypes : 0    {'Original': {}, 'Wavelet': {}, 'LoG': {}}
Name: diagnostics_Configuration_EnabledImageTypes, dtype: object
	 diagnostics_Image-original_Hash : 0    289b33d93939a924ef6f5df377fead1ef7786e2d
Name: diagnostics_Image-original_Hash, dtype: object
	 diagnostics_Image-original_Dimensionality : 0    3D
Name: diagnostics_

Name: wavelet-HLH_gldm_LargeDependenceLowGrayLevelEmphasis, dtype: object
	 wavelet-HLH_gldm_LowGrayLevelEmphasis : 0    0.6262157330892176
Name: wavelet-HLH_gldm_LowGrayLevelEmphasis, dtype: object
	 wavelet-HLH_gldm_SmallDependenceEmphasis : 0    0.005332330982901377
Name: wavelet-HLH_gldm_SmallDependenceEmphasis, dtype: object
	 wavelet-HLH_gldm_SmallDependenceHighGrayLevelEmphasis : 0    0.01332559387417819
Name: wavelet-HLH_gldm_SmallDependenceHighGrayLevelEmphasis, dtype: object
	 wavelet-HLH_gldm_SmallDependenceLowGrayLevelEmphasis : 0    0.0033340152600821737
Name: wavelet-HLH_gldm_SmallDependenceLowGrayLevelEmphasis, dtype: object
	 wavelet-HLH_glrlm_GrayLevelNonUniformity : 0    567941.9316203453
Name: wavelet-HLH_glrlm_GrayLevelNonUniformity, dtype: object
	 wavelet-HLH_glrlm_GrayLevelNonUniformityNormalized : 0    0.500000040276034
Name: wavelet-HLH_glrlm_GrayLevelNonUniformityNormalized, dtype: object
	 wavelet-HLH_glrlm_GrayLevelVariance : 0    0.24999997986198302
Name: w

	 log-sigma-5-0-mm-3D_glcm_Autocorrelation : 0    92.2771504069706
Name: log-sigma-5-0-mm-3D_glcm_Autocorrelation, dtype: object
	 log-sigma-5-0-mm-3D_glcm_ClusterProminence : 0    58.58892038705622
Name: log-sigma-5-0-mm-3D_glcm_ClusterProminence, dtype: object
	 log-sigma-5-0-mm-3D_glcm_ClusterShade : 0    1.7872878818972908
Name: log-sigma-5-0-mm-3D_glcm_ClusterShade, dtype: object
	 log-sigma-5-0-mm-3D_glcm_ClusterTendency : 0    1.6400120140620358
Name: log-sigma-5-0-mm-3D_glcm_ClusterTendency, dtype: object
	 log-sigma-5-0-mm-3D_glcm_Contrast : 0    0.040781476004080344
Name: log-sigma-5-0-mm-3D_glcm_Contrast, dtype: object
	 log-sigma-5-0-mm-3D_glcm_Correlation : 0    0.9513763380825216
Name: log-sigma-5-0-mm-3D_glcm_Correlation, dtype: object
	 log-sigma-5-0-mm-3D_glcm_DifferenceAverage : 0    0.03983482134788075
Name: log-sigma-5-0-mm-3D_glcm_DifferenceAverage, dtype: object
	 log-sigma-5-0-mm-3D_glcm_DifferenceEntropy : 0    0.23586534323289074
Name: log-sigma-5-0-mm-3D_glcm_

In [ ]:
def display_slices(image, mask, skip=1):
    """
    Displays a series of slices in z-direction that contains the segmented regions of interest.
    Ensures all contours are displayed in consistent and different colors.
        Parameters:
            image (array-like): Numpy array of image.
            mask (array-like): Numpy array of mask.
            skip (int): Only print every nth slice, i.e. if 3 only print every 3rd slice, default 1.
        Returns:
            None (series of in-line plots).
    """

    slice_locations = np.unique(np.where(mask != 0)[0]) # get indexes for where there is a contour present 
    slice_start = slice_locations[0] # first slice of contour 
    slice_end = slice_locations[len(slice_locations)-1] # last slice of contour
    
    counter = 1
    
    for img_arr, contour_arr in zip(image[slice_start:slice_end+1], mask[slice_start:slice_end+1]): # plot the slices with contours overlayed ontop
        if counter % skip == 0: # if current slice is divisible by desired skip amount 
            masked_contour_arr = np.ma.masked_where(contour_arr == 0, contour_arr)
            plt.imshow(img_arr, cmap='gray', interpolation='none')
            plt.imshow(masked_contour_arr, cmap='cool', interpolation='none', alpha=0.5, vmin = 1, vmax = np.amax(mask)) # vmax is set as total number of contours so same colors can be displayed for each slice
            plt.show()
        counter += 1

In [ ]:
n_slices_skip = 4
display_slices(image3D, gtv_mask, skip = n_slices_skip)